In [14]:
import os
import numpy as np
from tqdm.auto import tqdm
import mmcv
import matplotlib.pyplot as plt

In [128]:
keypoints = ["nose", "left_eye", "right_eye", "left_ear", "right_ear", "left_shoulder", "right_shoulder", "left_elbow", "right_elbow", "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee", "right_knee", "left_ankle", "right_ankle"]
my_keypoints = {"nose": "Nose",
                "left_ear": "Left Ear", 
                "right_ear": "Right Ear", 
                "left_shoulder": "Left Shoulder", 
                "right_shoulder": "Right Shoulder", 
                "left_elbow": "Left Elbow", 
                "right_elbow": "Right Elbow", 
                "left_wrist": "Left Wrist",
                "right_wrist": "Right Wrist",
                "left_hip": "Left Hip",
                "right_hip": "Right Hip",
                "left_knee": "Left Knee",
                "right_knee": "Right Knee",
                "left_ankle": "Left Ankle",
                "right_ankle": "Right Ankle"}

In [31]:

folder = '/home/tormod/ucph-erda-home/mmpose_results/'
video = 'IMG_2140'
path = f'{folder}{video}/'

def load_data(path):
    files = os.listdir(path)
    files.sort(key=lambda s: int(s.split('.')[0]))
    files = files[:500]
    no_files = len(files)
    data = np.zeros((no_files, 17, 3))
    for i, f in tqdm(enumerate(files), total=no_files):
        kp_frame = np.load(path + f, allow_pickle=True)
        if len(kp_frame) > 0:
            kp_frame = kp_frame[0]['keypoints']
            data[i] = kp_frame
    return data


In [32]:
data = load_data(path)


In [34]:

video = mmcv.video.VideoReader(f'videos/IMG_2140.MOV')

In [143]:
import xml.etree.ElementTree as ET


anno_file = '/home/tormod/Downloads/IMG_2140.xml'

def keypoint_str(keypoint):
    return f'{keypoint[0]:.2f},{keypoint[1]:.2f}'

def outside(keypoint, img_shape):
    return keypoint[0] < 0 or keypoint[1] < 0 or keypoint[0] >= img_shape[1] or keypoint[1] >= img_shape[0]

def to_cvat(anno_file, data):
    tree = ET.parse(anno_file)
    root = tree.getroot()
    size = root.find('meta/task/original_size')
    img_shape = int(size.find('height').text), int(size.find('width').text)
    for track_id in range(17):
        if track_id in [1,2]:
            continue
        track = ET.SubElement(root, 'track')
        track.set('id', str(track_id))
        track.set('label', my_keypoints[keypoints[track_id]])
        track.set('source', 'mmpose')
        for frame_id in range(data.shape[0]):
            point = ET.SubElement(track, 'points')
            keypoint = data[frame_id, track_id]
            point_str = keypoint_str(keypoint)
            point.set('frame', str(frame_id))
            point.set('outside', '1' if outside(keypoint, img_shape) else '0')
            point.set('occluded', '0')
            point.set('keyframe', '1')
            point.set('points', point_str)
            
    # Neck
    track_id = 18
    track = ET.SubElement(root, 'track')
    track.set('id', str(track_id))
    track.set('label', 'Neck')
    track.set('source', 'mmpose')
    for frame_id in range(data.shape[0]):
        point = ET.SubElement(track, 'points')
        keypoint = data[frame_id, 5:7, 0:2].mean(axis=0)
        point_str = keypoint_str(keypoint)
        point.set('frame', str(frame_id))
        point.set('outside', '1' if outside(keypoint, img_shape) else '0')
        point.set('occluded', '0')
        point.set('keyframe', '1')
        point.set('points', point_str)

    tree.write('test.xml')
    

In [144]:
to_cvat(anno_file, data)